In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity  
import itertools
from numpy import linalg as LA  
import os
print(os.listdir("E:\\Matlab\LinkPrediction\ml-latest-small"))

In [ ]:
A = pd.read_csv("E:/Matlab/LinkPrediction/ml-latest-small/ratings.csv")

In [ ]:
A.head()

In [ ]:
del A['rating']
#Split train and test sets based on the timeline. 依据时间戳来分训练集和测试集
timestamp90=A.timestamp.quantile(0.9)
train_split={'movieID':A.movieId[A.timestamp<timestamp90],'userID':A.userId[A.timestamp<timestamp90]}
test_split={'movieID':A.movieId[A.timestamp>=timestamp90],'userID':A.userId[A.timestamp>=timestamp90]}
train=pd.DataFrame(train_split)
train=train[['userID','movieID']]
train=train.reset_index(drop=True)
test=pd.DataFrame(test_split)
test=test[['userID','movieID']]
test=test.reset_index(drop=True)

In [ ]:
train.movieID[train.userID==583]

In [ ]:
#Check the number of users and movies 检查训练集里用户和影片的个数
user_node_num=train.userID.unique()
len(user_node_num)
user_node_num
movie_node_num=train.movieID.unique()
len(movie_node_num)

In [ ]:
#Set up network for training set 构筑训练集网络
movies=[]
user_movies=[]
for i in range(0,len(train.userID)):
        movies.append(train.movieID[i])
        if i+1 == len(train.movieID):
            user_movies.append([train.userID[i],movies])
        elif train.userID[i+1] != train.userID[i]:
            user_movies.append([train.userID[i],movies])
            movies=[]

In [ ]:
#Set up user list for training set建立训练集里用户ID列表
train_user_list=[]
for i in range(0,len(user_movies)):
    train_user_list.append(user_movies[i][0])
#Covert to matrix转化为矩阵
train_user_array=np.asarray(train_user_list)

In [ ]:
train_movie_list=[]
for i in train.movieID:
    if i not in train_movie_list:
        train_movie_list.append(i)
train_movie_list=sorted(train_movie_list)
train_movie_array=np.asarray(train_movie_list)

In [ ]:
# Set up User-Movie Sparse Matrix 建造用户-电影的稀疏矩阵
user_movie_summary=[]
for w in range(0,len(user_movies)):
    for i in range(0,len(train_movie_list)):
        if train_movie_list[i] in user_movies[w][1]:
            user_movie_summary.append(1)
        else:
            user_movie_summary.append(0)

In [ ]:
# Change the data type 将矩阵转换为数据框 
user_movie_summary=np.asarray(user_movie_summary)
user_movie_summary=user_movie_summary.reshape(len(user_movies),len((train_movie_list)))
user_movie_summary=pd.DataFrame(user_movie_summary)

In [ ]:
# Similarity matrix among users based on CN 基于共同邻居，创建用户-用户相似性矩阵
user2user = np.zeros(shape=(user_movie_summary.shape[0],user_movie_summary.shape[0]))
u=[]
for h in range(0,user_movie_summary.shape[1]):
    for i in range(0,user_movie_summary.shape[0]):
        if user_movie_summary[h][i]==1:
            u.append(i)
        if i+1 == user_movie_summary.shape[0]:
            similar_users=list(itertools.combinations(u,2))
            u=[]
            for w in range(0,len(similar_users)):
                user2user[similar_users[w][0]][similar_users[w][1]]=user2user[similar_users[w][0]][similar_users[w][1]]+1

In [ ]:
# Symmertic Matrix 让用户-用户矩阵变为对称矩阵
user2user=user2user+user2user.T

In [ ]:
# Similarity matrix among users based on cosine 基于余弦相似性，建立用户-用户相似性矩阵
user_similarity=cosine_similarity(user_movie_summary) 

In [ ]:
# Similarity matrix among movies based on cosine 基于余弦相似性，建立电影-电影相似性矩阵
movie_similarity=cosine_similarity(user_movie_summary.T) 

In [ ]:
#Similarity matrix among movies based on CN 基于共同邻居，建立电影-电影相似性矩阵
movie2movie = np.zeros(shape=(user_movie_summary.T.shape[0],user_movie_summary.T.shape[0]))
m=[]
for h in range(0,user_movie_summary.T.shape[1]):
    for i in range(0,user_movie_summary.T.shape[0]):
        if user_movie_summary.T[h][i]==1:
            m.append(i)
        if i+1 == user_movie_summary.T.shape[0]:
            similar_movies=list(itertools.combinations(m,2))
            m=[]
            for w in range(0,len(similar_movies)):
                movie2movie[similar_movies[w][0]][similar_movies[w][1]]=movie2movie[similar_movies[w][0]][similar_movies[w][1]]+1

In [ ]:
# Symmertic Matrix 让电影-电影矩阵变为对称矩阵
movie2movie=movie2movie+movie2movie.T

In [ ]:
# Generate the recommended scores based on CN(users)基于用户间的共同邻居指标来获取推荐列表，及列表中每部电影的推荐分数
from operator import itemgetter
ums_matrix=np.asarray(user_movie_summary)
def recommenduserCN(ID):
    unwatched = np.where(ums_matrix[np.where(train_user_array==ID)[0][0]]==0)
    scores=[]
    sim=0
    sim_sum=sum(user2user[np.where(train_user_array==ID)[0][0]])
    for i in range(0,len(unwatched[0])):
        others_watched=np.where(ums_matrix[:,unwatched[0][i]]==1) 
        for w in others_watched[0]:
            sim += user2user[np.where(train_user_array==ID)[0][0]][w] 
            if w == others_watched[0][-1]:
                scores.append([train_movie_array[unwatched[0][i]],sim/sim_sum]) 
                sim = 0
    sort_unwatched=sorted(scores, key=itemgetter(1),reverse=True)
    return sort_unwatched

# Generate the recommended scores based on cosine(users) 基于用户间的余弦相似性指标来获取推荐列表，及列表中每部电影的推荐分数
def recommenduserCOS(ID):
    unwatched = np.where(ums_matrix[np.where(train_user_array==ID)[0][0]]==0)
    scores=[]
    sim=0
    sim_sum=sum(user_similarity[np.where(train_user_array==ID)[0][0]])
    for i in range(0,len(unwatched[0])):
        others_watched=np.where(ums_matrix[:,unwatched[0][i]]==1) 
        for w in others_watched[0]:
            sim += user_similarity[np.where(train_user_array==ID)[0][0]][w] 
            if w == others_watched[0][-1]:
                scores.append([train_movie_array[unwatched[0][i]],[sim/sim_sum]])
                sim = 0
    sort_unwatched=sorted(scores, key=itemgetter(1),reverse=True)
    return sort_unwatched

In [ ]:
# Generate the recommended scores based on CN(movies) 基于电影间的共同邻居指标来获取推荐列表，及列表中每部电影的推荐分数
def recommenditemCN(ID):
    unwatched = np.where(ums_matrix[np.where(train_user_array==ID)[0][0]]==0) 
    other_movie_watched=np.where(ums_matrix.T[:,np.where(train_user_array==ID)[0][0]]==1)
    scores=[]
    sim=0
    for i in range(0,len(unwatched[0])): 
        for w in range(0,len(other_movie_watched)): 
            sim += movie2movie[unwatched[0][i]][other_movie_watched[0][w]]
            if w+1 == len(other_movie_watched):
                scores.append([train_movie_array[unwatched[0][i]],[sim/sum(movie2movie[unwatched[0][i]])]])
                sim = 0
    sort_unwatched=sorted(scores, key=itemgetter(1),reverse=True)
    return sort_unwatched

# Generate the recommended scores based on cosine(movies)基于电影间的共同邻居指标来获取推荐列表，及列表中每部电影的推荐分数
def recommenditemCOS(ID):
    unwatched = np.where(ums_matrix[np.where(train_user_array==ID)[0][0]]==0) 
    other_movie_watched=np.where(ums_matrix.T[:,np.where(train_user_array==ID)[0][0]]==1)
    sim=0
    for i in range(0,len(unwatched[0])): 
        for w in range(0,len(other_movie_watched)): 
            sim += movie_similarity[unwatched[0][i]][other_movie_watched[0][w]]
            if w+1 == len(other_movie_watched):
                scores.append([train_movie_array[unwatched[0][i]],[sim/sum(movie_similarity[unwatched[0][i]])]])
                sim = 0
    sort_unwatched=sorted(scores, key=itemgetter(1),reverse=True)
    return sort_unwatched

In [ ]:
# Find users in both training and testing sets同时存在于训练集和测试集的用户
common_users=sorted(list(set(train.userID)&set(test.userID)))

In [ ]:
# Evaluate the precision of recommendation list CN method(users) 计算以用户间的共同邻居为基础的推荐准确率 长度20
precision_list=[]
for i in common_users:
        precision=len(list(set(test.movieID[test.userID==i])& set([item[0] for item in recommenduserCN(i)][:20])))/len([item[0]for item in recommenduserCN(i)][:20])
        precision_list.append(precision)
precision_user_CN = np.mean(precision_list)

In [ ]:
# Evaluate the precision of recommendation list Cosine method(users)计算以用户间的余弦相似性为基础的推荐准确率 长度20
precision_list=[]
for i in common_users:
        precision=len(list(set(test.movieID[test.userID==i])& set([item[0] for item in recommenduserCOS(i)][:20])))/len([item[0]for item in recommenduserCOS(i)][:20])
        precision_list.append(precision)
precision_user_COS = np.mean(precision_list)

In [ ]:
precision_user_CN

In [ ]:
precision_user_COS

In [ ]:
# Evaluate the rank score CN method (users)计算以用户间的共同邻居为基础的排序准确度
user_RS_list=[]
all_RS_list=[]
for i in common_users:
    rec= recommenduserCN(i)
    for w in  list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])):
        for z in range(len(rec)):
            if w==rec[z][0]:
                RS=(z+1)/len(rec)
                user_RS_list.append(RS)
                if list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])).index(w)+1 \
                == len(list(set(test.movieID[test.userID==i])& set([item[0] for item in rec]))):
                    user_RS=np.mean(user_RS_list)
                    all_RS_list.append([i,user_RS]
                    user_RS_list=[]
                break
RS_user_CN=(np.mean([piece[1] for piece in all_RS_list]))

In [ ]:
RS_user_CN

In [ ]:
# Evaluate the precision Cosine method (users) 计算以用户间的余弦相似性为基础的推荐准确率
precision_list=[]
for i in common_users:
        precision=len(list(set(test.movieID[test.userID==i])& set([item[0] for item in recommenduserCOS(i)][:20])))/len([item[0]for item in recommenduserCOS(i)][:20])
        precision_list.append(precision)
precision_user_COS = np.mean(precision_list)

In [ ]:
# Evaluate the rank score Cosine method (users)计算以用户间的余弦相似性为基础的排序准确度 
user_RS_list=[]
all_RS_list=[]
for i in common_users:
    rec= recommenduserCOS(i)
    for w in  list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])):
        for z in range(len(rec)):
            if w==rec[z][0]: 
                RS=(z+1)/len(rec)
                user_RS_list.append(RS)
                if list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])).index(w)+1 \
                == len(list(set(test.movieID[test.userID==i])& set([item[0] for item in rec]))):
                    user_RS=np.mean(user_RS_list)
                    all_RS_list.append([i,user_RS])
                    user_RS_list=[]
                break
RS_user_COS=(np.mean([piece[1] for piece in all_RS_list]))

In [ ]:
RS_user_COS

In [ ]:
# Evaluate the precision of recommendation list CN method(movies)计算以电影间的共同邻居为基础的推荐准确率
precision_list=[]
for i in common_users:
        precision=len(list(set(test.movieID[test.userID==i])& set([item[0] for item in recommenditemCN(i)][:20])))/len([item[0]for item in recommenditemCN(i)][:20])
        precision_list.append(precision)
precision_movie_CN = np.mean(precision_list)

In [ ]:
# Evaluate the rank score CN method (movies)计算以电影间的共同邻居为基础的排序准确度
user_RS_list=[]
all_RS_list=[]
for i in common_users:
    rec= recommenditemCN(i)
    for w in  list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])):
        for z in range(len(rec)):
            if w==rec[z][0]:
                RS=(z+1)/len(rec)
                user_RS_list.append(RS)
                if list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])).index(w)+1 \
                == len(list(set(test.movieID[test.userID==i])& set([item[0] for item in rec]))):
                    user_RS=np.mean(user_RS_list)
                    all_RS_list.append([i,user_RS])#38号用户没有看过一部推荐列表的电影，在列表里就没有录入该用户，是否会影响最后RS的准确性？
                    user_RS_list=[]
                break
RS_movie_CN=(np.mean([piece[1] for piece in all_RS_list]))

In [ ]:
# Evaluate the precision Cosine method (movies) 计算以电影间的余弦相似性为基础的推荐准确率
precision_list=[]
for i in common_users:
        precision=len(list(set(test.movieID[test.userID==i])& set([item[0] for item in recommenditemCOS(i)][:20])))/len([item[0]for item in recommenditemCOS(i)][:20])
        precision_list.append(precision)
precision_movie_COS = np.mean(precision_list)

In [ ]:
# # Evaluate the rank score Cosine method (movies) 计算以电影间的余弦相似性为基础的排序准确度
user_RS_list=[]
all_RS_list=[]
for i in common_users:
    rec= recommenditemCOS(i)
    for w in  list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])):
        for z in range(len(rec)):
            if w==rec[z][0]:
                RS=(z+1)/len(rec)
                user_RS_list.append(RS)
                if list(set(test.movieID[test.userID==i])& set([item[0] for item in rec])).index(w)+1 \
                == len(list(set(test.movieID[test.userID==i])& set([item[0] for item in rec]))):
                    user_RS=np.mean(user_RS_list)
                    all_RS_list.append([i,user_RS])#38号用户没有看过一部推荐列表的电影，在列表里就没有录入该用户，是否会影响最后RS的准确性？
                    user_RS_list=[]
                break
RS_movie_COS=(np.mean([piece[1] for piece in all_RS_list]))

In [ ]:
print("\n The precision score based on user(CN) is: {:.5f}".format(precision_user_CN))
print("\n The precision score based on user(COSINE) is: {:.5f}".format(precision_user_COS))
print("\n The precision score based on movie(CN) is: {:.5f}".format(precision_movie_CN))
print("\n The precision score based on movie(COSINE) is: {:.5f}".format(precision_movie_COS))
print("\n The ranking score based on user(CN) is: {:.5f}".format(RS_user_CN))
print("\n The ranking score based on user(COSINE) is: {:.5f}".format(RS_user_COS))
print("\n The ranking score based on movie(CN) is: {:.5f}".format(RS_movie_CN))
print("\n The ranking score based on movie(COSINE) is: {:.5f}".format(RS_movie_COS))